In [2]:
# import dependencies

import os, io, requests, zipfile
import itertools
import datetime
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime, time
from sklearn.metrics.pairwise import cosine_similarity as cos_sig

from geopy.distance import great_circle
plt.rcParams['figure.figsize'] = 10,6
import warnings
warnings.filterwarnings("ignore")

In [3]:
def download_ais_data(year, month, zone, data_dir='./data'):
    '''function to download ais data from https://marinecadastre.gov/ais/ 
    and return corresponding pandas dataframe'''
    
    # create data directory
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # create path to csv file
    csv_file = 'AIS_{}_{}_Zone{}.csv'.format(year, str(month).zfill(2), str(zone).zfill(2))
    csv_path = os.path.join(data_dir, 'AIS_ASCII_by_UTM_Month', str(year), csv_file)
        
    try:
        # read csv if already downloaded
        data = pd.read_csv(csv_path)
    except:
        # create zip file url
        zip_file_url = ('https://coast.noaa.gov/htdata/CMSP/AISDataHandler/'
                        '{}/AIS_{}_{}_Zone{}.zip'.format(year, year, 
                                                         str(month).zfill(2), 
                                                         str(zone).zfill(2)))
        # download and extract data
        r = requests.get(zip_file_url, stream=True)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(data_dir)

        # load csv as dataframe
        data = pd.read_csv(csv_path)

    return data

dfLASD = download_ais_data(2017,6,11)
print(dfLASD.shape)

(11341146, 16)


In [4]:
tugs = [21,22,31,32,52,1023,1025]
dfLASD = dfLASD[~dfLASD['VesselType'].isin(tugs)]
print(dfLASD.shape)
dfLASD.head()

(10019514, 16)


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo
0,338108787,2017-06-01T00:00:00,34.40449,-119.69123,0.0,63.1,511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,338067800,2017-06-01T00:00:26,33.98221,-118.45072,0.0,-133.4,511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,338131199,2017-06-01T00:00:07,33.75583,-118.11642,0.0,-78.8,511.0,NaN,NaN,NaN,NaN,undefined,NaN,NaN,NaN,NaN
3,338103724,2017-06-01T00:00:46,32.71513,-117.56661,10.0,88.6,511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,338092107,2017-06-01T00:00:59,33.72581,-118.07638,0.0,57.1,511.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfLASD['BaseDateTime'] = pd.to_datetime(dfLASD['BaseDateTime'], errors='raise')
dfLASD['date'] = dfLASD.BaseDateTime.apply(lambda x: x.date())


In [6]:
minTime = min(dfLASD['BaseDateTime'])
dfLASD['SecTime'] = dfLASD.BaseDateTime.apply(lambda x: time.mktime(x.timetuple()) - time.mktime(minTime.timetuple()))

In [6]:
# %%time
# interval = 150  #time interval (seconds) 
# distance = 24000 #24,000feet = 8000 yards 
# # neighbor_vessels = pd.DataFrame(columns= dfLASD.columns)

# df1 = dfLASD.sort_values(by='SecTime', ascending=True) #this is the dataframe to look for matches within
# df1 = df1.dropna(how='any') 
# df1 = df1.iloc[:200] #subsetting to 2000 rows for proof of concept

# # for i,v in enumerate(df1.BaseDateTime.values): 
    
# # #     df2 = df1.iloc[i+1:,] #removing rows we've already iterated through
# # #     time_min = v - pd.Timedelta(seconds=interval)
# # #     colArr = np.array([None]*16)
# #     time_max = v + pd.Timedelta(seconds=interval) 
# # #     temp_df = df2[(df2.BaseDateTime > time_min)&(df2.BaseDateTime < time_max)] #makes temp df of datapoints close in time
# #     for ii, vv in enumerate(df1.iloc[i+2]['BaseDateTime'].values):
# #         if(df1.iloc[ii]['BaseDateTime'] <= time_max):
# #             neighbor_vessels = neighbor_vessels.append(df1.iloc[ii])
# #         else:
# #             break
            
    
        
    
# #     #getting data into format to use geopy distance calculator
# #     temp_df['latlon'] = list(zip(temp_df.LAT, temp_df.LON)) 
# #     temp_df['dist'] = temp_df['latlon'].apply(lambda x: great_circle(temp_df.latlon.values[0], x).feet) #calc distances from row 0
    
# #     closers = temp_df[(np.abs(temp_df.dist) < distance)] #filter points close in time
# #     closers = closers.drop_duplicates(subset ='MMSI') #drop duplicates (same boat)
    
# #     if len(closers) > 1: #if closers includes more than one boat
# #         neighbor_vessels = neighbor_vessels.append(closers)


# for i, v in enumerate(df1.SecTime.values):
#     time_max = v + interval
#     for ii, vv in enumerate(df1.SecTime):
#         if vv <= time_max:
#                 neighbor_vessels = df1[['MMSI', 'BaseDateTime']].iloc[ii]
#         else:
#             break

In [7]:
# %%time
# for i, v in enumerate(df1.SecTime.values):
#     time_max = v + interval
#     for ii, vv in enumerate(df1.SecTime):
#         if vv <= time_max:
#                 neighbor_vessels = df1[['MMSI', 'BaseDateTime']].iloc[i]
#         else:
#             break

# neighbor_vessels = [df1[['MMSI', 'BaseDateTime']].iloc[i] for i, v in enumerate(df1.SecTime.values) for ii, vv in enumerate(df1.SecTime) if vv <= (v + interval)]


# # print(neighbor_vessels[0])

In [8]:
# test = np.array(range(0,20))
# print(test)
# i = 5
# for ii in range(i+1, len(test)):
#     print(ii)

In [34]:
%%time
di = 0
days = 1
day_groups = dfLASD.groupby(['date'])
for name, group in day_groups:
    print(name)
    print(group.shape)
    if(di >= days):
        break
    di += 1
    dfDay = group.copy()
    minSec = dfDay['SecTime'].min()
    maxSec = dfDay['SecTime'].max()
    minLat = dfDay['LAT'].min()
    maxLat = dfDay['LAT'].max()
    minLon = dfDay['LON'].min()
    maxLon = dfDay['LON'].max()
    dfDay['NormSecTime'] = (dfDay['SecTime'] - minSec) / (maxSec - minSec) * (0.01 / (150 / (maxSec - minSec)))
    dfDay['NormLat'] = (dfDay['LAT'] - minLat) / (maxLat - minLat) * (0.01 / (0.1 / (maxLat - minLat)))
    dfDay['NormLon'] = (dfDay['LON'] - minLon) / (maxLon - minLon) * (0.01 / (0.065 / (maxLon - minLon)))
    mxDay = dfDay[['NormSecTime', 'NormLat', 'NormLon']].iloc[:8000].values 
    d = np.matmul(mxDay,mxDay.transpose())
    for i in range(len(mxDay)):
        if(i % (int(len(mxDay)/ 10)) == 0):
            print(i/8000)
        A = np.linalg.norm(mxDay[i,:])
        for j in range(len(mxDay.transpose()[0])):
            B = np.linalg.norm(mxDay.transpose()[:,j])
            d[i,j] = d[i,j] / (A * B)

print(d)

2017-06-01
(299047, 18)
0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
2017-06-02
(322742, 18)
[[ 1.          0.99099162  0.98513782 ...,  0.99171536  0.99329811
   0.99284613]
 [ 0.99099162  1.          0.99926629 ...,  0.97968047  0.98120765
   0.98076522]
 [ 0.98513782  0.99926629  1.         ...,  0.97284601  0.97436821
   0.97392523]
 ..., 
 [ 0.99171536  0.97968047  0.97284601 ...,  1.          0.99991189
   0.99995636]
 [ 0.99329811  0.98120765  0.97436821 ...,  0.99991189  1.          0.99999226]
 [ 0.99284613  0.98076522  0.97392523 ...,  0.99995636  0.99999226  1.        ]]
CPU times: user 6min 54s, sys: 6.11 s, total: 7min
Wall time: 7min 10s


In [71]:
dfL = pd.read_csv('data/AIS_LA_SD_Jan_1_to_15_2016_Filtered_by_Proximity.csv')
dfRes = pd.DataFrame(columns = dfL.columns)

In [81]:
r = len(d)
clrgArr = []
for i in range(r):
    if(i % (int(r/ 10)) == 0):
           print(i/8000)
    for j in range(i+1,r):
        if(d[i][j] >= 0.99):
            clrgArr.append((i,j))
#     df
print(clrgArr[:5])

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
[(0, 1), (0, 12), (0, 19), (0, 20), (0, 36)]


0


In [122]:
%%time
cs = ['MMSI','BaseDateTime','LAT','LON','SOG','COG','Heading',
     'VesselName','IMO','CallSign','VesselType','Status','Length',
     'Width','Draft','Cargo']
shipsArr = np.array([None] * 33)

for x in range(10):
    i = clrgArr[x][0]
    j = clrgArr[x][1]
    if dfLASD.at[i,'MMSI'] != dfLASD.at[j,'MMSI']:
        ship1 = dfLASD[cs].iloc[i].values
#         print(ship1[0])
        ship2 = dfLASD[cs].iloc[j].values
        shipsArr[0] = x
        shipsArr[1:17] = ship1
        shipsArr[17:] = ship2
#         print(shipsArr)
        dfRes = dfRes.append(pd.Series(shipsArr), ignore_index = True)


CPU times: user 9.82 s, sys: 13 s, total: 22.8 s
Wall time: 28.1 s


In [103]:
print(dfRes.head())

   ID  MMSI_1  Timestamp_1  LAT_1  LON_1  SOG_1  COG_1  Heading_1  \
0 NaN     NaN          NaN    NaN    NaN    NaN    NaN        NaN   
1 NaN     NaN          NaN    NaN    NaN    NaN    NaN        NaN   
2 NaN     NaN          NaN    NaN    NaN    NaN    NaN        NaN   
3 NaN     NaN          NaN    NaN    NaN    NaN    NaN        NaN   
4 NaN     NaN          NaN    NaN    NaN    NaN    NaN        NaN   

   Vessel Name_1  IMO_1 ...     23        24   25   26      27  \
0            NaN    NaN ...  511.0       NaN  NaN  NaN     NaN   
1            NaN    NaN ...  511.0       NaN  NaN  NaN     NaN   
2            NaN    NaN ...  511.0       NaN  NaN  NaN     NaN   
3            NaN    NaN ...  511.0       NaN  NaN  NaN     NaN   
4            NaN    NaN ...  511.0  MARE'ZIA  NaN  NaN  1019.0   

                       28     29   30  31  32  
0                     NaN    NaN  NaN NaN NaN  
1                     NaN    NaN  NaN NaN NaN  
2  under way using engine    NaN  NaN NaN Na

In [35]:
print(0.01 / (0.1 / (maxLat - minLat)))      
print(0.01 / (0.065 / (maxLon - minLon)))
print(0.01 / (150 / (maxSec - minSec)))


2.9575
0.89216
5.75993333333


In [96]:
print(dfLASD.columns)

Index(['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',
       'VesselName', 'IMO', 'CallSign', 'VesselType', 'Status', 'Length',
       'Width', 'Draft', 'Cargo', 'date', 'SecTime'],
      dtype='object')


In [95]:
a = [1,2,3,4,5,6,7]
print(a[1:4])
print(a[4:])

[2, 3, 4]
[5, 6, 7]


In [97]:
c = ['MMSI','LAT','LON','SOG','COG','Heading',
     'VesselName','IMO','CallSign','VesselType','Status','Length',
     'Width','Draft','Cargo']
print(len(c))

15


In [116]:
print(len(clrgArr))

22906783
